# AutoARIMA/SARIMA

**Choice of model:**

- To assess whether Bitcoin prices exhibit autocorellation, we decided to fit the logarithmic returns of Bitcoin using the ARIMA/SARIMA models.

- For SARIMA, we also wanted to check if there is any seasonality in the data (although Bitcoin is traded 24 hours a day 7 days a week, we were curious to see if there is any changes in the price action pattern due to the concentration of larger instituional players, or weekends).




In [ ]:
!pip -q install statsforecast utilsforecast pandas matplotlib scikit-learn

import numpy as np
import matplotlib.pyplot as plt

from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA

from utilsforecast.evaluation import evaluate
from utilsforecast.losses import mae,rmse

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.6/354.6 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.7/280.7 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd


## **✔Model Execution ✅**

The SARIMA model was fit using the StatsForecast AutoARIMA function. A rolling forecast was used for cross-validation, and MAE and RMSE were computed for each fold.

In [ ]:
!pip -q install statsforecast utilsforecast pandas matplotlib scikit-learn

import numpy as np
import matplotlib.pyplot as plt

from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA

from utilsforecast.evaluation import evaluate
from utilsforecast.losses import mae,rmse


In [ ]:
df=pd.read_csv("bitcoin_2018.csv")

df.head()

,date,btc_open,btc_high,btc_low,btc_close,btc_volume,fng_value,fng_class,active_addresses,tx_count,...,bitcoin_interest,dollar_value,10Y_yield,cpi,dow_adj,sp_adj,nasdaq_adj,sp_ret,nasdaq_ret,dow_ret
0,2018-01-02,13189.08,14780.26,12910.58,14596.11,13013.672931,30.0,Fear,850802.0,340980.0,...,51.0,109.6444,2.46,248.859,24824.009766,2695.810059,7006.899902,0.008303,0.014994,0.004239
1,2018-01-03,14596.68,15500.00,14546.28,14834.30,14242.678887,30.0,Fear,937531.0,395963.0,...,51.0,109.6751,2.44,248.859,24922.679688,2713.060059,7065.529785,0.006399,0.008367,0.003975
2,2018-01-04,14822.12,15430.27,14192.37,14780.40,14155.248510,30.0,Fear,1054712.0,425008.0,...,51.0,109.4779,2.46,248.859,25075.130859,2723.989990,7077.910156,0.004029,0.001752,0.006117
3,2018-01-05,14781.45,16532.99,14773.67,16367.42,15989.730597,30.0,Fear,888403.0,342707.0,...,51.0,109.3496,2.47,248.859,25295.869141,2743.149902,7136.560059,0.007034,0.008286,0.008803
4,2018-01-06,16367.42,17200.00,16194.98,16772.83,11893.846940,30.0,Fear,889099.0,358847.0,...,51.0,109.3496,2.47,248.859,25295.869141,2743.149902,7136.560059,0.007034,0.008286,0.008803


We choose to study log returns of Bitcoin instead of studying the spot for the following reasons:
- Bitcoin prices follow exponential growth, and ARIMA / SARIMA are primarily designed to handle linear data;
- To improve the stationarity of the data;
- Finally, it makes MSE and MAE more interpretable.



In [ ]:
# ======================================================================
#     Create dataframe df for data manipulation for ARIMA / SARIMA
# ======================================================================
# We label the columns correctly for the model to be able to run
# We also define the logarithmic returns where appropriate, as discussed above
# Note that exogenous values will need to be set up individually for each model


file_path="bitcoin_2018.csv"
series_id="BTC"
y_column="btc_open_log"


print(f"Loading data from: {file_path}")
df = pd.read_csv(file_path)
# Rename the target as y
df.rename(columns={y_column : "y"}, inplace=True)

# Create unique_id
df["unique_id"]=series_id
# Create ds column with datetime values
df["ds"] = pd.to_datetime(df["date"]).copy()

# --- Calculate the missing daily return ---
# Daily Return Formula: (Close Price at T - Close Price at T-1) / Close Price at T-1
df['btc_daily_return'] = df['btc_close'].pct_change()


# Define the list of NUMERIC features for which logarithmic returns will be computed and used for modeling
# Here, the selection is done manually based on following characteristics:
# Selected features correspond to continuous data in units,
# are prone to changing variance, and/or exhibit exponential returns.



features_to_log=[
        'btc_open', 'btc_high', 'btc_low',
        'btc_close', 'btc_volume',
        'active_addresses', 'tx_count',
        'tx_volume', 'dollar_value',
        'dow_adj',	'sp_adj',	'nasdaq_adj'
        ]



# 1. Create LOG RETURN features (X) for prediction (T DATA)
for col in features_to_log:
    # Create a new column with the suffix '_log'
    df[f'{col}_log'] = np.log(df[col]).diff()

# 2. Rename the target as y
df.rename(columns={y_column : "y"}, inplace=True)

    # Drop rows with NaN (last N_DAYS rows due to future return)
    # Filter for columns ending in 'log'
log_features = [col for col in df.columns if f'_log' in col]

df= df.dropna(subset=log_features)




Loading data from: bitcoin_2018.csv


In [ ]:
# Print the columns of interest to check
df[["ds", "y"]].head()

,ds,y
1,2018-01-03,0.101405
2,2018-01-04,0.015327
3,2018-01-05,-0.002748
4,2018-01-06,0.101920
5,2018-01-07,0.023643


In [ ]:
# ============================================================
#     ARIMA / SARIMA with cross validation : Log returns
# ============================================================



# 1) Finalize set up of the dataframe for ARIMA / SARIMA analysis
df_forecast=df.copy()
df_forecast = df_forecast[["ds", "y", "unique_id"]].copy()


# 2) Define Cross-Validation settings
h = 30            # forecast horizon: 30 days
n_windows = 8     # last 8 windows (in other words, "number of folds")
SEASON_LENGTH = 7 # length of the seasonal period

# 3) Define SARIMA model
models_arima = [
    AutoARIMA(seasonal=False, alias="ARIMA"),
    AutoARIMA(season_length=SEASON_LENGTH, alias="SARIMA")
]

sf_arima = StatsForecast(models=models_arima, freq="D")

# 4) Perform rolling time-series cross-validation
cv_df_arima = sf_arima.cross_validation(
    df=df_forecast,
    h=h,
    n_windows=n_windows,
    step_size=h,
    refit=True
)


# 5) Evaluate MAE and RMSE across all windows and series
cv_eval_arima = evaluate(
    cv_df_arima.drop(columns=["cutoff"]),
    metrics=[mae, rmse],
)


# 6) Average over all windows for each metric
cv_summary_arima = (
    cv_eval_arima
    .drop(columns=["unique_id"])
    .groupby("metric", as_index=False)
    .mean()
)

display(cv_summary_arima)

,metric,ARIMA,SARIMA
0,mae,0.014283,0.01422
1,rmse,0.019246,0.01920


In [ ]:
# =============================================================
#         ARIMA / SARIMA with cross validation : Log returns
# =============================================================

# REMARK: The difference between this ARIMA / SARIMA models and the previous one
# is that we use more windows (16 windows as opposed to 8).

# 1) Finalize set up of the dataframe for ARIMA / SARIMA analysis

df_forecast = df_forecast[["ds", "y", "unique_id"]].copy()


# 2) Define Cross-Validation settings
h = 30              # forecast horizon: 30 days
n_windows = 16     # last 16 windows (in other words, "number of folds")
SEASON_LENGTH = 7 # length of the seasonal period

# 3) Define SARIMA model
models_arima = [
    AutoARIMA(seasonal=False, alias="ARIMA"),
    AutoARIMA(season_length=SEASON_LENGTH, alias="SARIMA")
]

sf_arima = StatsForecast(models=models_arima, freq="D")

# 4) Perform rolling time-series cross-validation
cv_df_arima = sf_arima.cross_validation(
    df=df_forecast,
    h=h,
    n_windows=n_windows,
    step_size=h,
    refit=True
)


# 5) Evaluate MAE and RMSE across all windows and series
cv_eval_arima = evaluate(
    cv_df_arima.drop(columns=["cutoff"]),
    metrics=[mae, rmse],
)


# 6) Average over all windows for each metric
cv_summary_arima = (
    cv_eval_arima
    .drop(columns=["unique_id"])
    .groupby("metric", as_index=False)
    .mean()
)

display(cv_summary_arima)

# Interpretation: Despite increasing the number of windows for cross-validation, we see that the MAE and RMSE increased:
# This suggests that increasing the number of windows added potentially added noisier segments,
# or it could be that the added windows were under a different regime.

,metric,ARIMA,SARIMA
0,mae,0.016641,0.016597
1,rmse,0.023098,0.023053


In [ ]:

# =============================================================
#         ARIMA / SARIMA with cross validation : Log returns
# =============================================================

# REMARK: Follozing the previous observatio that increasing the number of windows increased MSE and RMSE,
# we choose to fix n_windows at 8.
# The difference between this ARIMA / SARIMA models and the very first ones in this notebook
# is that we truncate the dataset starting from January 2024:
# Potentially, the autocorellation between current and more recent Bitcoin prices could be stronger
# due to the market conditions and players being more comparable than during the earlier periods.

df_forecast = df_forecast[["ds", "y", "unique_id"]].copy()
df_forecast=df_forecast[df_forecast["ds"]>"01-01-2024"]


# 1) Define Cross-Validation settings
h = 30              # forecast horizon: 30 days
n_windows = 8     # last 8 weekly windows (in other words, "number of folds")
SEASON_LENGTH = 7 # length of the seasonal period

# 2) Define SARIMA model
models_arima = [
    AutoARIMA(seasonal=False, alias="ARIMA"),
    AutoARIMA(season_length=SEASON_LENGTH, alias="SARIMA")
]

sf_arima = StatsForecast(models=models_arima, freq="D")

# 3) Perform rolling time-series cross-validation
cv_df_arima = sf_arima.cross_validation(
    df=df_forecast,
    h=h,
    n_windows=n_windows,
    step_size=h,
    refit=True
)


# 4) Evaluate MAE and RMSE across all windows and series
cv_eval_arima = evaluate(
    cv_df_arima.drop(columns=["cutoff"]),
    metrics=[mae, rmse],
)


# Average over all windows for each metric
cv_summary_arima = (
    cv_eval_arima
    .drop(columns=["unique_id"])
    .groupby("metric", as_index=False)
    .mean()
)

display(cv_summary_arima)


# Interpretation: the MAE and RMSE is very consistent with that for the model fitted on the entire dataset starting from 2018.

,metric,ARIMA,SARIMA
0,mae,0.014243,0.014209
1,rmse,0.019304,0.019246


In [ ]:
# =================================================
#         ARIMA / SARIMA with exogenous variable
# =================================================

# Finally, to check whether our relatively high MAE and RMSE can be explained
# by effects of exogenous variables, we add features to our model.

# 1) Build dataframe with target and exogenous 'temp'
df_exog_cv=df.copy()

exog_columns=['fng_value', 'bitcoin_interest', 'tx_count_log', 'sp_adj_log', 'cpi']

df_exog_cv = (
    df_exog_cv[["unique_id", "ds", "y"]+exog_columns]
    .dropna(subset=(["ds", "y"]+exog_columns))
    .sort_values(["unique_id", "ds"])
    .reset_index(drop=True)
)

# Separate the part used as "y" and the part used as exogenous X
df_exog_y = df_exog_cv[["unique_id", "ds", "y"]].copy()
X_exog    = df_exog_cv[["unique_id", "ds"]+exog_columns].copy()

# 2) Define cross-validation settings (same as for SARIMA)
h = 30
n_windows = 8

# 3) Define exogenous models
models_exog = [
    AutoARIMA(seasonal=False, alias="ARIMA_exog"),
    AutoARIMA(season_length=SEASON_LENGTH, alias="SARIMA_exog")
]

sf_exog = StatsForecast(models=models_exog, freq="D")

# 4) Perform rolling time-series cross-validation with exogenous variable

cv_df_exog = sf_exog.cross_validation(
    df=df_exog_cv,
    h=h,
    n_windows=n_windows,
    step_size=h,
    refit=True
)


# 5) Evaluate MAE and RMSE across all windows and series
cv_eval_exog = evaluate(
    cv_df_exog.drop(columns=["cutoff"]),
    metrics=[mae, rmse],
)



# 7) Average over windows (and series) per metric
cv_summary_exog = (
    cv_eval_exog
    .drop(columns=["unique_id"])
    .groupby("metric")
    .mean()
    .reset_index()
)

display(cv_summary_exog)

# Interpretation: Although the MAE and RMSE have slightly decreased compared,
# which is expected given that we are adding more variables to the model,
# the change appears to be rather marginal, and overall MAE and RMSE remain elevated.

,metric,ARIMA_exog,SARIMA_exog
0,mae,0.013998,0.014010
1,rmse,0.018676,0.018681
